In [3]:
import duckdb
import pandas as pd

duckdb.sql("""
    CREATE OR REPLACE TABLE orders AS 
    SELECT 
        "Customer ID" AS customer_id,
        "Customer Status" AS customer_status,
        "Order ID" AS order_id,
        "Product ID" AS product_id,
        "Cost Price Per Unit" AS cost_price_per_unit,
        "Total Retail Price for This Order" AS total_retail_price,
        "Quantity Ordered" AS quantity_ordered,
        "Date Order was placed" AS order_date,
        "Delivery Date" AS delivery_date
    FROM read_csv_auto('orders.csv');
""")

duckdb.sql("""
    CREATE OR REPLACE TABLE product_supplier AS 
    SELECT 
        "Product ID" AS product_id,
        "Product Category" AS product_category,
        "Supplier Name" AS supplier_name,
        "Product Name" AS product_name,
        "Product Line" AS product_line,
        "Product Group" AS product_group,
        "Supplier Country" AS supplier_country
    FROM read_csv_auto('product-supplier.csv');
""")



In [4]:
duckdb.sql("SELECT * FROM orders LIMIT 5;").df()


,customer_id,customer_status,order_id,product_id,cost_price_per_unit,total_retail_price,quantity_ordered,order_date,delivery_date
0,579,Silver,123002578,220101400106,20.70,92.6,2,01-Jan-17,07-Jan-17
1,7574,SILVER,123004074,210201000009,9.95,21.7,1,01-Jan-17,05-Jan-17
2,28861,Gold,123000871,230100500068,0.80,1.7,1,01-Jan-17,04-Jan-17
3,43796,Gold,123002851,220100100633,24.05,47.9,1,01-Jan-17,06-Jan-17
4,54673,Gold,123003607,220200200043,18.30,36.9,1,01-Jan-17,04-Jan-17


In [5]:
duckdb.sql("SELECT * FROM product_supplier LIMIT 5;").df()


,product_id,product_category,supplier_name,product_name,product_line,product_group,supplier_country
0,210100100001,Children Outdoors,Scandinavian Clothing A/S,Boy's and Girl's Ski Pants with Braces,Children,"Outdoor things, Kids",NO
1,210100100002,Children Outdoors,Luna sastreria S.A.,Children's Jacket,Children,"Outdoor things, Kids",ES
2,210100100003,Children Outdoors,Scandinavian Clothing A/S,Children's Jacket Sidney,Children,"Outdoor things, Kids",NO
3,210100100004,Children Outdoors,Scandinavian Clothing A/S,Children's Rain Set,Children,"Outdoor things, Kids",NO
4,210100100005,Children Outdoors,Scandinavian Clothing A/S,Children's Rain Suit,Children,"Outdoor things, Kids",NO


Query : 1
Retrieve all customer records
Extract the full dataset of customers without any filters or limitations.

In [6]:
duckdb.sql("""
SELECT DISTINCT customer_id, customer_status
FROM orders
""").df()

,customer_id,customer_status
0,79782,Silver
1,11065,GOLD
2,22890,SILVER
3,85474,Gold
4,3699,Gold
...,...,...
111636,71503,GOLD
111637,92156,Gold
111638,17699,SILVER
111639,28324,Silver


Query : 2
Select customers from a specified location
Identify and isolate customers based on their geographic location or city.


In [23]:
duckdb.sql("""
SELECT DISTINCT o.customer_id, p.supplier_country
FROM orders o
JOIN product_supplier p ON o.product_id = p.product_id
WHERE p.supplier_country = 'SE'
""").df()

,customer_id,supplier_country
0,54801,SE
1,10700,SE
2,31228,SE
3,11278,SE
4,236,SE
...,...,...
3862,26155,SE
3863,19043,SE
3864,34450,SE
3865,59796,SE


Query : 3
Extract essential product details
Retrieve only product name, category, and price for a simplified product overview.

In [8]:
duckdb.sql("""
SELECT product_name, product_category
FROM product_supplier
""").df()

,product_name,product_category
0,Boy's and Girl's Ski Pants with Braces,Children Outdoors
1,Children's Jacket,Children Outdoors
2,Children's Jacket Sidney,Children Outdoors
3,Children's Rain Set,Children Outdoors
4,Children's Rain Suit,Children Outdoors
...,...,...
5499,Top Equipe 00 White,Winter Sports
5500,Top Equipe 07 Green,Winter Sports
5501,Top Equipe 99 Black,Winter Sports
5502,Alpine Ski Bag 2-pair Black/Yellow,Winter Sports


Query : 4
List products sorted by price, highest first
Show the product catalog with the most expensive items at the top.


In [9]:
duckdb.sql("""
SELECT p.product_name, MAX(o.total_retail_price) AS max_price
FROM product_supplier p
JOIN orders o ON p.product_id = o.product_id
GROUP BY p.product_name
ORDER BY max_price DESC
""").df()

,product_name,max_price
0,Top-form 325 Treadmill,6382.00
1,Fit4you Ski Jacket Astro,3740.00
2,Letour Trimag Bike,3708.60
3,Family Holiday 4,3599.40
4,Letour Heart Bike,3199.20
...,...,...
3119,Long Life,4.80
3120,Universal,3.80
3121,"Toe Brake Block, Black",3.10
3122,Yellow,2.50


Query : 5
Show orders combined with respective customer details
Join orders with their corresponding customer information.


In [10]:
duckdb.sql("""
SELECT o.order_id, o.customer_id, o.product_id, o.quantity_ordered, o.total_retail_price, o.order_date,
       p.product_name, p.product_category
FROM orders o
JOIN product_supplier p ON o.product_id = p.product_id
""").df()

,order_id,customer_id,product_id,quantity_ordered,total_retail_price,order_date,product_name,product_category
0,123952566,35967,230100100012,2,376.4,16-Jun-20,Jacket Talkeetna,Outdoors
1,123955644,37582,240700200021,2,39.2,16-Jun-20,Helmet XL,Team Sports
2,123951077,37734,240100100463,1,14.6,16-Jun-20,Sparkle Spray Orange,Assorted Sports Articles
3,123955655,38047,240500200088,2,79.4,16-Jun-20,Maxrun Long-sleeved T-Shirt,Running - Jogging
4,123959163,39347,220100100154,2,157.6,16-Jun-20,Big Guy Men's Full Zip Micro Fibre Jacket,Clothes
...,...,...,...,...,...,...,...,...
185008,123955599,34659,210200500015,3,53.4,16-Jun-20,Teen Lifa Ski Undershirt,Children Sports
185009,123959057,35243,230100600026,2,237.2,16-Jun-20,"Expedition 20,Large,Right,Forestgreen",Outdoors
185010,123951073,35403,210200300051,1,25.7,16-Jun-20,Tony's Core Mini Bag,Children Sports
185011,123959073,35864,230100600022,5,420.5,16-Jun-20,"Expedition10,Medium,Right,Blue Ribbon",Outdoors


Query : 6
Calculate total quantity sold for each product
Sum all quantities ordered for every product.


In [11]:
duckdb.sql("""
SELECT p.product_name, SUM(o.quantity_ordered) AS total_quantity_sold
FROM orders o
JOIN product_supplier p ON o.product_id = p.product_id
GROUP BY p.product_name
""").df()

,product_name,total_quantity_sold
0,"White 90,Top-Flite Aero 3-pack",50.0
1,Petanque Balls Chromium 8-pack,1700.0
2,Hgf Tarantula Men's Indoor Shoes,156.0
3,Proplay Women's Tour Force 7w,82.0
4,Big Guy Men's Victory Peach Poplin Pants,97.0
...,...,...
3119,"Clipper Comp,Rottbeer Fade",1.0
3120,Tick Ski Turtleneck w/Zipper,4.0
3121,Osprey Kien Lieu Pro,7.0
3122,Top Granite Men's Shorts,8.0


Query : 7
Compute total revenue generated by each product
Aggregate sales revenue per product.

In [12]:
duckdb.sql("""
SELECT p.product_name, SUM(o.total_retail_price) AS total_revenue
FROM orders o
JOIN product_supplier p ON o.product_id = p.product_id
GROUP BY p.product_name
""").df()

,product_name,total_revenue
0,Woman's Clima Fit Ultra Lite Jacket,10721.40
1,Pants Fun,7481.80
2,Profleece with Zipper,14182.50
3,"Expedition10,Medium,Right,Blue Ribbon",87716.30
4,Sunfit Madly Swim Suit,11494.20
...,...,...
3119,Swoosh Cap,146.70
3120,Big Guy Men's Tactel Pants w/Piping,364.00
3121,Grey PSU,137.47
3122,"Shuttlecocks, Gold",37.40


Query : 8
Identify products that achieved revenue beyond a certain limit
Filter and list products that generated substantial revenue.

In [13]:
duckdb.sql("""
SELECT p.product_name, SUM(o.total_retail_price) AS total_revenue
FROM orders o
JOIN product_supplier p ON o.product_id = p.product_id
GROUP BY p.product_name
HAVING SUM(o.total_retail_price) > 50000
""").df()

,product_name,total_revenue
0,Power Dmx Wide Shoes,77163.800
1,Expedition Dome 3,275760.000
2,Big Guy Men's Air Tuned Sirocco Shoes,176262.600
3,Feelgood 55-75 Litre Black Women's Backpack,61763.700
4,Petanque Balls Chromium 8-pack,89570.500
...,...,...
58,Down Jacket,50175.000
59,Hurricane 4,331534.900
60,"Twain X-Scream 7.9 Ski,Sq 750 Dri",64371.625
61,"Expedition 20,Medium,Right,Forestgreen",84322.400


Query : 9
Create a ranked list of customers based on spending
Order customers from highest to lowest total purchase amount.


In [14]:
duckdb.sql("""
SELECT customer_id, SUM(total_retail_price) AS total_spent,
       RANK() OVER (ORDER BY SUM(total_retail_price) DESC) AS rank
FROM orders
GROUP BY customer_id
""").df()

,customer_id,total_spent,rank
0,7766,6826.300,1
1,31519,6585.800,2
2,77062,6569.600,3
3,89591,6432.500,4
4,54290,6382.000,5
...,...,...,...
56022,52859,1.300,56022
56023,2079,1.300,56022
56024,3985,1.300,56022
56025,87416,0.625,56026


Query : 10
Generate monthly revenue summaries segmented by product category
Show how revenue varies month-wise for each product category.

In [15]:
duckdb.sql("""
SELECT strftime('%Y-%m', strptime(order_date, '%d-%b-%y')) AS month,
       p.product_category,
       SUM(o.total_retail_price) AS monthly_revenue
FROM orders o
JOIN product_supplier p ON o.product_id = p.product_id
GROUP BY month, p.product_category
ORDER BY month, p.product_category;
""").df()

,month,product_category,monthly_revenue
0,2017-01,Assorted Sports Articles,41594.675
1,2017-01,Children Sports,9340.700
2,2017-01,Clothes,42914.950
3,2017-01,Golf,13128.800
4,2017-01,Indoor Sports,15421.000
...,...,...,...
715,2021-12,Running - Jogging,31874.080
716,2021-12,Shoes,156687.710
717,2021-12,Swim Sports,4970.800
718,2021-12,Team Sports,23348.380


Query : 11
Find customers whose spending is above the average customer’s spend
Determine which customers spend more than the overall average.

In [16]:
duckdb.sql("""
WITH customer_total AS (
    SELECT customer_id, SUM(total_retail_price) AS total_spent
    FROM orders
    GROUP BY customer_id
),
average_spend AS (
    SELECT AVG(total_spent) AS avg_spent
    FROM customer_total
)
SELECT customer_id, total_spent
FROM customer_total
WHERE total_spent > (SELECT avg_spent FROM average_spend)
""").df()

,customer_id,total_spent
0,64570,504.79
1,65471,1320.50
2,78633,1345.00
3,90860,1463.64
4,19546,2208.80
...,...,...
20033,30165,514.20
20034,38917,719.50
20035,8628,780.30
20036,34969,754.20


Query : 12
Assign customers into spending brackets (High, Medium, Low)
Categorize customers based on total spend.

In [17]:
duckdb.sql("""
WITH customer_total AS (
    SELECT customer_id, SUM(total_retail_price) AS total_spent
    FROM orders
    GROUP BY customer_id
)
SELECT customer_id, total_spent,
       CASE 
           WHEN total_spent >= 50000 THEN 'High'
           WHEN total_spent >= 20000 THEN 'Medium'
           ELSE 'Low'
       END AS spending_bracket
FROM customer_total
""").df()

,customer_id,total_spent,spending_bracket
0,37582,474.60,Low
1,37734,1309.40,Low
2,56358,346.20,Low
3,63858,18.10,Low
4,70339,1280.20,Low
...,...,...,...
56022,81822,34.50,Low
56023,88,248.20,Low
56024,1934,43.98,Low
56025,8162,44.10,Low


Query : 13
Identify customers who made multiple purchases over time
Spot repeat buyers and analyze purchase frequency

In [18]:
duckdb.sql("""
SELECT customer_id, COUNT(DISTINCT order_id) AS num_orders
FROM orders
GROUP BY customer_id
HAVING COUNT(DISTINCT order_id) > 1
""").df()

,customer_id,num_orders
0,7973,4
1,23755,2
2,8507,3
3,22896,2
4,73952,3
...,...,...
41014,33123,2
41015,18699,2
41016,73715,2
41017,41017,2


Query : 14
Analyze customer retention by grouping customers based on their first purchase month
Track how many customers make repeat purchases in later months.


In [19]:
duckdb.sql("""
WITH first_purchase AS (
    SELECT customer_id, MIN(strftime('%Y-%m', strptime(order_date, '%d-%b-%y'))) AS first_month
    FROM orders
    GROUP BY customer_id
)
SELECT f.first_month, COUNT(DISTINCT o.customer_id) AS repeat_customers
FROM orders o
JOIN first_purchase f ON o.customer_id = f.customer_id
WHERE strftime('%Y-%m', strptime(o.order_date, '%d-%b-%y')) > f.first_month
GROUP BY f.first_month
""").df()

,first_month,repeat_customers
0,2020-02,383
1,2017-01,2093
2,2018-08,848
3,2018-05,1215
4,2018-12,585
5,2019-02,766
6,2018-04,1273
7,2018-03,1147
8,2018-10,564
9,2017-08,1254


Query : 15
Detect product pairs that are frequently bought together
Identify bundles or commonly purchased combinations.

In [20]:
duckdb.sql("""
SELECT o1.product_id AS product_1, o2.product_id AS product_2, COUNT(*) AS times_bought_together
FROM orders o1
JOIN orders o2 ON o1.order_id = o2.order_id AND o1.product_id < o2.product_id
GROUP BY o1.product_id, o2.product_id
ORDER BY times_bought_together DESC
LIMIT 20
""").df()

,product_1,product_2,times_bought_together


Query : 16
Calculate and interpret moving averages of daily sales data 
Smooth sales trends using a rolling window.

In [21]:
duckdb.sql("""
SELECT strptime(order_date, '%d-%b-%y') AS order_date_parsed,
       SUM(total_retail_price) AS daily_sales,
       AVG(SUM(total_retail_price)) OVER (ORDER BY strptime(order_date, '%d-%b-%y') ROWS BETWEEN 6 PRECEDING AND CURRENT ROW) AS moving_avg_7days
FROM orders
GROUP BY order_date_parsed
ORDER BY order_date_parsed
""").df()

,order_date_parsed,daily_sales,moving_avg_7days
0,2017-01-01,225.500,225.500000
1,2017-01-02,10476.300,5350.900000
2,2017-01-03,8683.200,6461.666667
3,2017-01-04,3434.190,5704.797500
4,2017-01-05,9129.800,6389.798000
...,...,...,...
1821,2021-12-27,34508.340,26907.890000
1822,2021-12-28,37662.285,27045.540000
1823,2021-12-29,13634.500,25813.540000
1824,2021-12-30,34466.430,26438.295714


Query : 17
Produce monthly product sales rankings within each category, including ties
Rank products by sales for each category and month.


In [22]:
duckdb.sql("""
SELECT strftime('%Y-%m', strptime(order_date, '%d-%b-%y')) AS month,
       p.product_category,
       p.product_name,
       SUM(o.quantity_ordered) AS total_quantity,
       RANK() OVER (PARTITION BY strftime('%Y-%m', strptime(order_date, '%d-%b-%y')), p.product_category
                    ORDER BY SUM(o.quantity_ordered) DESC) AS rank
FROM orders o
JOIN product_supplier p ON o.product_id = p.product_id
GROUP BY month, p.product_category, p.product_name
ORDER BY month, p.product_category, rank
""").df()

,month,product_category,product_name,total_quantity,rank
0,2017-01,Assorted Sports Articles,Bulls Eye Stuart/Tungsten 24 Gram,17.0,1
1,2017-01,Assorted Sports Articles,Aim4it 16 Gram Softtip Pil,14.0,2
2,2017-01,Assorted Sports Articles,Aim4it 18 Gram Softtip Pil,10.0,3
3,2017-01,Assorted Sports Articles,Brt Anorak,7.0,4
4,2017-01,Assorted Sports Articles,Petanque Balls Chromium 8-pack,7.0,4
...,...,...,...,...,...
93082,2021-12,Winter Sports,Tempest Jacket,3.0,44
93083,2021-12,Winter Sports,"Ii Pmt,Bone",3.0,44
93084,2021-12,Winter Sports,Men's Ski Jacket Vent Air,3.0,44
93085,2021-12,Winter Sports,Truls Pants Snowboard Hollofil,3.0,44
